In [1]:
import pandas as pd
import torch.utils.data as Data
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import init
import matplotlib.pyplot as plt
from torch import nn

In [401]:
HIDDEN_LAYERS = 10
B_INIT = -0.2
ACTIVATION = F.relu
class FcNet(torch.nn.Module):
    def __init__(self, num_features, dim_out, batch_normalization=True):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(FcNet, self).__init__()
        self.do_bn = batch_normalization
        self.fcs = []
        self.bns = []
        self.drops = []
        self.bn_input = nn.BatchNorm1d(num_features, momentum=0.5)   # for input data
        self.dropout_input = nn.Dropout()
        for i in range(HIDDEN_LAYERS):
            input_size = num_features if i == 0 else 50 # each hidden layer has 10 neurons
            fc = nn.Linear(input_size, 50)
            setattr(self, 'fc%i' % i, fc) # such that pytorch can distinguish different layers
            self._set_init(fc)
            self.fcs.append(fc)
            if self.do_bn:
                bn = nn.BatchNorm1d(50, momentum=0.5)
                setattr(self, 'bn%i' % i, bn)
                self.bns.append(bn)
            fc = nn.Dropout()
            self.drops.append(fc)
            setattr(self, 'drop%i' %i, fc)                
        self.predict = nn.Linear(50, 1)
        self._set_init(self.predict)
    
    def _set_init(self, layer):
        init.normal(layer.weight, mean=0, std=0.1)
        init.constant(layer.bias, B_INIT)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        pre_activation = [x]
        if self.do_bn: x = self.bn_input(x)
        layer_input = [x]
        for i in range(HIDDEN_LAYERS):
            x = self.fcs[i](x)
            pre_activation.append(x)
            if self.do_bn:
                x = self.bns[i](x)
            x = ACTIVATION(x)
            x = self.drops[i](x)
            layer_input.append(x)
        out = self.predict(x)
        return out, layer_input, pre_activation

In [374]:
# df = pd.read_csv('route16.csv', index_col=0)
# no_dirty_cols = ['TT', 'flow','holiday','hldy_seq','weekday','timeslot',
#                  'occ1', 'occ2', 'occ3', 'occ4',
#                  'Spd1','Spd2','Spd3','Spd4']
# df[no_dirty_cols].to_csv('route16_1603.csv')

In [235]:
df = pd.read_csv('route15.csv', index_col=0)

In [431]:
train_df = df.copy()

In [ ]:
# 0.0499 with bs=288, shuffle=True

In [390]:
# feature set 1
#train_df['TT_prv2'] = df['TT'].shift(2)
#train_df['TT_prv3'] = df['TT'].shift(3)
#train_df['flow_prv2'] = df['flow'].shift(3)
#train_df['flow_prv3'] = df['flow'].shift(3)
train_df[['occ1_prv2', 'occ2_prv2', 'occ3_prv2', 'occ4_prv2']] = train_df[['occ1', 'occ2', 'occ3', 'occ4']].shift(2)
train_df[['occ1_prv3', 'occ2_prv3', 'occ3_prv3', 'occ4_prv3']] = train_df[['occ1', 'occ2', 'occ3', 'occ4']].shift(3)
train_df[['Spd1_prv2','Spd2_prv2','Spd3_prv2','Spd4_prv2']] = train_df[['Spd1','Spd2','Spd3','Spd4']].shift(2)
train_df[['Spd1_prv3','Spd2_prv3','Spd3_prv3','Spd4_prv3']] = train_df[['Spd1','Spd2','Spd3','Spd4']].shift(3)
train_df[['flow1_prv2','flow2_prv2','flow3_prv2','flow4_prv2']] = train_df[['flow1','flow2','flow3','flow4']].shift(2)
train_df[['flow1_prv3','flow2_prv3','flow3_prv3','flow4_prv3']] = train_df[['flow1','flow2','flow3','flow4']].shift(3)
train_df[['W_TT_prv2',]] = train_df[['W_TT',]].shift(2)
train_df[['W_TT_prv3',]] = train_df[['W_TT',]].shift(3)
train_df[['S_TT_prv2',]] = train_df[['S_TT',]].shift(2)
train_df[['S_TT_prv3',]] = train_df[['S_TT',]].shift(3)
train_df[['L_TT_prv2',]] = train_df[['L_TT',]].shift(2)
train_df[['L_TT_prv3',]] = train_df[['L_TT',]].shift(3)

drop_cols = ['TT', 'flow','year','month',
            'occ1', 'occ2', 'occ3', 'occ4',
            'Spd1','Spd2','Spd3','Spd4','Spd5',
            'flow1','flow2','flow3','flow4',
            'W_TT', 'S_TT', 'L_TT']
for i in train_df.columns:
    try:
        if  ('Avg' in i or
            'Trnd' in i or
            'Last' in i or
            'Ramp' in i or
            'Closed' in i or
            'Lanes' in i or
            'lanes' in i or
            '5' in i or
            'sVF' == i or
            'bVF' == i or
            'holiday' == i or
            'JT' in i or
            'FT' in i):
            drop_cols.append(i)        
        if (i.split('_')[1].startswith('nxt') or 
            i.split('_')[1].endswith('mean') or
            i.split('_')[1].endswith('dev') or
            i.split('_')[1].startswith('chng') or
            i.split('_')[1].endswith('prv1')):
            #print(i)
            drop_cols.append(i)

    except IndexError:
        pass
    
train_df = train_df.drop(drop_cols, axis=1)

In [349]:
# feature set 2
def condition(s):
    for i in ['SpdAvg', 'SpdTrnd', 'occ', 'Evt', 'Lanes'] + ['W_TT','S_TT']: # 0.056
    #for i in ['SpdAvg', 'SpdTrnd', 'occ', 'Evt', 'Lanes'] + ['W_TT','S_TT']: # 0.059
        if s.startswith(i):
            return True
    return False
keep_cols = [col for col in df.columns if condition(col)]
keep_cols += ['hldy_seq', 'weekday', 'timeslot', 'TT_mean']
train_df = df[keep_cols]

In [275]:
# feature set 3
#keep_cols = ['hldy_seq', 'weekday', 'timeslot', 'TT_mean','W_TT','S_TT']

In [276]:
# feature set 4
"""
drop_columns = ['Closed2','Closed3','Closed4','Closed5','Ramp0','Ramp1','Ramp2','Ramp3','Ramp4','Ramp5',
                'Evt4','Evt5','Lanes0','Lanes1','Lanes2','Lanes3','Lanes4','Lanes5','Closed0','Closed1',
                'SpdTrnd3','density3','SpdTrnd4','density4','SpdTrnd5','density5','Evt0','Evt1','Evt2','Evt3',
                'FT5','flow5','occ5','lanes5','SpdTrnd0','density0','SpdTrnd1','density1','SpdTrnd2','density2',
                'SpdLast4','JT4','FT4','flow4','occ4','lanes4','Spd5','SpdAvg5','SpdLast5','JT5',
                'Spd3','SpdAvg3','SpdLast3','JT3','FT3','flow3','occ3','lanes3','Spd4','SpdAvg4',
                'occ1','lanes1','Spd2','SpdAvg2','SpdLast2','JT2','FT2','flow2','occ2','lanes2',
                'FT0','flow0','occ0','lanes0','Spd1','SpdAvg1','SpdLast1','JT1','FT1','flow1',
                'Spd0','SpdAvg0','SpdLast0','JT0','TT_nxt3','TT_nxt6','TT_nxt12','W_TT','S_TT','L_TT','TT_chng1',
                'TT_mean','flow_mean','flow_chng1','flow_chng2']

keep_columns = ['holiday','hldy_seq','weekday','timeslot',
                'TT_prv2','TT_prv3',
                'flow_prv2','flow_prv3']

train_df = df[keep_columns]
"""

"\ndrop_columns = ['Closed2','Closed3','Closed4','Closed5','Ramp0','Ramp1','Ramp2','Ramp3','Ramp4','Ramp5',\n                'Evt4','Evt5','Lanes0','Lanes1','Lanes2','Lanes3','Lanes4','Lanes5','Closed0','Closed1',\n                'SpdTrnd3','density3','SpdTrnd4','density4','SpdTrnd5','density5','Evt0','Evt1','Evt2','Evt3',\n                'FT5','flow5','occ5','lanes5','SpdTrnd0','density0','SpdTrnd1','density1','SpdTrnd2','density2',\n                'SpdLast4','JT4','FT4','flow4','occ4','lanes4','Spd5','SpdAvg5','SpdLast5','JT5',\n                'Spd3','SpdAvg3','SpdLast3','JT3','FT3','flow3','occ3','lanes3','Spd4','SpdAvg4',\n                'occ1','lanes1','Spd2','SpdAvg2','SpdLast2','JT2','FT2','flow2','occ2','lanes2',\n                'FT0','flow0','occ0','lanes0','Spd1','SpdAvg1','SpdLast1','JT1','FT1','flow1',\n                'Spd0','SpdAvg0','SpdLast0','JT0','TT_nxt3','TT_nxt6','TT_nxt12','W_TT','S_TT','L_TT','TT_chng1',\n                'TT_mean','flow_mean','flow_chng1','

In [432]:
# FEATURE set 5, teacher's
train_df['TT_prv3'] = train_df['TT'].shift(3)
train_df['flow_prv3'] = train_df['flow'].shift(3)
train_df['TT_prv2'] = train_df['TT'].shift(2)
keep_cols = ['W_TT','S_TT','SpdAvg0','occ0','SpdAvg1','occ1','SpdAvg2','occ2','SpdAvg3','occ3',
'SpdAvg4','occ4','SpdAvg5','occ5','SpdTrnd0','SpdTrnd1','SpdTrnd2','SpdTrnd3','SpdTrnd4','SpdTrnd5',
'Evt0','Evt1','Evt2','Evt3','Evt4','Evt5','Lanes0','Lanes1','Lanes2','Lanes3','Lanes4','Lanes5',
'hldy_seq','weekday','timeslot','TT_mean']
train_df = df[keep_cols]

In [438]:
from datetime import datetime, timedelta

In [439]:
val_start_date, val_end_date = '20170701', '20170731' # include
train_start_date, train_end_date = '20170301', '20170630'

In [440]:
def get_data_loader(mins='30', batch_size=288):
    target = pd.DataFrame(df['TT'])
    target['TT_30min_later'] = target['TT'].shift(periods = -6) # 30min
    target['TT_60min_later'] = target['TT'].shift(periods = -12) # 60min
    train_entries = (datetime.strptime(train_end_date, '%Y%m%d') - datetime.strptime(train_start_date, '%Y%m%d')).days * 288 # days * 5mins
    val_entries = (datetime.strptime(val_end_date, '%Y%m%d') - datetime.strptime(val_start_date, '%Y%m%d')).days * 288
    start_index = df.index.get_loc(datetime.strptime(train_start_date, '%Y%m%d').strftime('%Y-%m-%d %H:%M:%S'))
    
    x_train, x_val, x_test = train_df[start_index: start_index+train_entries].fillna(-99), train_df[start_index+train_entries:start_index+train_entries+val_entries].fillna(-99), train_df[start_index+train_entries+val_entries:].fillna(-99)
    if mins=='30':
        y_train, y_val, y_test = target[start_index: start_index+train_entries]['TT_30min_later'].fillna(-99), target[start_index+train_entries:start_index+train_entries+val_entries]['TT_30min_later'].fillna(-1), target[start_index+train_entries+val_entries:]['TT_30min_later'].fillna(-1)
    elif mins=='60':
        y_train, y_val, y_test = target[start_index: start_index+train_entries]['TT_60min_later'].fillna(-99), target[start_index+train_entries:start_index+train_entries+val_entries]['TT_60min_later'].fillna(-1), target[start_index+train_entries+val_entries:]['TT_60min_later'].fillna(-1)
    x_size = x_train.shape[1]
    y_size = 1 # for regression
    x_train, y_train = torch.from_numpy(x_train.values).float(), torch.from_numpy(y_train.values).float()
    x_val, y_val = Variable(torch.from_numpy(x_val.values).float()).cuda(), Variable(torch.from_numpy(y_val.values).float()).cuda()

    train_dataset = Data.TensorDataset(data_tensor=x_train, target_tensor=y_train)
    train_loader = Data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2,)
    return train_loader, x_val, y_val

In [441]:
def mape_loss(predict, target, to_numpy=False, model=None, in_=None):
    import numpy
    assert ((type(predict) == type(target) == torch.autograd.variable.Variable) or
            type(predict) == type(target) == numpy.ndarray), 'check the type of inputs, they have to be all torch Variable or numpy ndarray'
    if type(predict) == type(target) == torch.autograd.variable.Variable:
        if to_numpy == True:
            import numpy as np
            loss = (np.sum(np.abs((target.type(torch.FloatTensor).data.numpy().flatten() - 
                                  model(in_)[0].type(torch.FloatTensor).data.numpy().flatten())) 
                          / target.type(torch.FloatTensor).data.numpy().flatten()) 
                    / len(target))
            return loss
        loss = (1 / predict.shape[0] * 
               torch.sum((target - predict).abs() / target))
    elif type(predict) == type(target) == numpy.ndarray:
        loss = (1 / predict.shape[0] * 
               numpy.sum(numpy.abs(target.flatten() - predict.flatten()) / target.flatten()))   
    return loss

In [445]:
epoch = 300

data_loader30, x_val30, y_val30 = get_data_loader('30')
data_loader60, x_val60, y_val60 = get_data_loader('60')
x_size = data_loader30.dataset.data_tensor.shape[1]
y_size = 1
model_30 = FcNet(x_size, y_size)
model_60 = FcNet(x_size, y_size)

model_30.cuda() # using gpu
model_60.cuda()
#loss_func = torch.nn.MSELoss()
loss_func = mape_loss
#optimizer = torch.optim.SGD(model.parameters(), lr=0.03)
optimizer30 = torch.optim.Adam(model_30.parameters(), lr=0.03)
optimizer60 = torch.optim.Adam(model_60.parameters(), lr=0.03)

def train(train_loader, x_val, y_val, optimizer, model):
    model.train()
    for t in range(epoch):  
        for step, (b_x, b_y) in enumerate(train_loader):
            b_x, b_y = Variable(b_x).type(torch.FloatTensor).cuda(), Variable(b_y).type(torch.FloatTensor).cuda()
            pred, _, _ = model(b_x)
            loss = loss_func(pred, b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        y_val_predict, _, _ = model(x_val)
        print(str(t+1))
        print('training error:', loss.data[0])
        print('validation error:', loss_func(y_val_predict, y_val).data[0])
        
def test(x, model):
    model.eval()
    return model(x)

In [446]:
train(data_loader30, x_val30, y_val30, optimizer30, model_30)
predict30 = test(Variable(data_loader30.dataset.data_tensor).type(torch.FloatTensor).cuda())[0].type(torch.FloatTensor).data.numpy()
#predict30 = model_30(Variable(data_loader30.dataset.data_tensor).type(torch.FloatTensor).cuda())[0].type(torch.FloatTensor).data.numpy()
ground_truth30 = data_loader30.dataset.target_tensor.numpy()

1
training error: 45.26845169067383
validation error: 1527.093505859375
2
training error: 42.34086227416992
validation error: 1419.624755859375
3
training error: 35.6618537902832
validation error: 1370.709716796875
4
training error: 37.768028259277344
validation error: 1386.120849609375
5
training error: 42.09209442138672
validation error: 1355.6956787109375
6
training error: 35.78422546386719
validation error: 1356.6842041015625
7
training error: 35.539031982421875
validation error: 1327.346923828125
8
training error: 37.563106536865234
validation error: 1328.2225341796875
9
training error: 36.99211502075195
validation error: 1316.5069580078125
10
training error: 35.9259033203125
validation error: 1300.5262451171875
11
training error: 37.21657180786133
validation error: 1295.78369140625
12
training error: 38.12016677856445
validation error: 1279.537841796875
13
training error: 33.68669891357422
validation error: 1285.895751953125
14
training error: 33.48235321044922
validation error: 

113
training error: 17.6699275970459
validation error: 848.4420166015625
114
training error: 16.66347312927246
validation error: 848.2479858398438
115
training error: 16.991600036621094
validation error: 848.6133422851562
116
training error: 17.715190887451172
validation error: 848.3154907226562
117
training error: 16.670127868652344
validation error: 848.3075561523438
118
training error: 17.92780303955078
validation error: 848.5970458984375
119
training error: 15.80156135559082
validation error: 848.3954467773438
120
training error: 19.702970504760742
validation error: 848.4456787109375
121
training error: 16.49020004272461
validation error: 848.3814086914062
122
training error: 16.904396057128906
validation error: 848.3941040039062
123
training error: 16.125438690185547
validation error: 848.4378051757812
124
training error: 16.114116668701172
validation error: 848.3145141601562
125
training error: 18.454587936401367
validation error: 848.6415405273438
126
training error: 20.87705230

224
training error: 17.11248207092285
validation error: 848.3926391601562
225
training error: 15.42059326171875
validation error: 848.393310546875
226
training error: 16.777400970458984
validation error: 848.4684448242188
227
training error: 18.823659896850586
validation error: 848.405517578125
228
training error: 17.43711280822754
validation error: 848.281494140625
229
training error: 16.618026733398438
validation error: 848.4427490234375
230
training error: 15.202309608459473
validation error: 848.1633911132812
231
training error: 18.873064041137695
validation error: 848.4461059570312
232
training error: 17.397783279418945
validation error: 848.45849609375
233
training error: 16.324214935302734
validation error: 848.3816528320312
234
training error: 17.640975952148438
validation error: 848.4329833984375
235
training error: 18.806808471679688
validation error: 848.3241577148438
236
training error: 17.016530990600586
validation error: 848.4324951171875
237
training error: 17.1735477447

TypeError: test() missing 1 required positional argument: 'model'

In [255]:
train(data_loader60, x_val60, y_val60, optimizer60, model_60)
predict60 = model_60(Variable(data_loader30.dataset.data_tensor).type(torch.FloatTensor).cuda())[0].type(torch.FloatTensor).data.numpy()
ground_truth60 = data_loader60.dataset.target_tensor.numpy()

1
training error: 1.5691022872924805
validation error: 860.8607788085938
2
training error: 2.13673734664917
validation error: 882.0859985351562
3
training error: 1.4838981628417969
validation error: 853.4680786132812
4
training error: 1.240260124206543
validation error: 848.103759765625
5
training error: 1.5227031707763672
validation error: 872.0254516601562
6
training error: 1.669386863708496
validation error: 864.37451171875
7
training error: 1.8556016683578491
validation error: 875.1535034179688
8
training error: 1.6123440265655518
validation error: 855.876953125
9
training error: 1.8330744504928589
validation error: 868.1317138671875
10
training error: 1.7927443981170654
validation error: 867.0516967773438
11
training error: 2.2954394817352295
validation error: 849.0803833007812
12
training error: 1.5145777463912964
validation error: 851.0097045898438
13
training error: 2.218322277069092
validation error: 844.5438232421875
14
training error: 2.394016981124878
validation error: 854.

In [271]:
import matplotlib.pyplot as plt


def save_figs(predict, ground_truth, x_val, y_val, mins='30'):
    x = range(288)
    if mins == '30':
        val_predict = model_30(x_val.type(torch.FloatTensor).cuda())[0].type(torch.FloatTensor).data.numpy().flatten()
    elif mins == '60':
        val_predict = model_60(x_val.type(torch.FloatTensor).cuda())[0].type(torch.FloatTensor).data.numpy().flatten()
    else:
        print('??')
        return
    val_ground_truth = y_val.type(torch.FloatTensor).data.numpy().flatten()    
    for i in range(0, len(predict), 288):
        plt.figure(figsize=(8, 6))
        p1, = plt.plot(x, predict[i:i+288], 'r', label='predict')
        p2, = plt.plot(x, ground_truth[i:i+288], 'b', label='target')
        plt.legend(handles=[p1, p2])
        title = df.index[i].split()[0]
        plt.title(title)
        if mins=='30':
            plt.savefig('figs/30/'+title+'.jpg')
        elif mins=='60':
            plt.savefig('figs/60/'+title+'.jpg')
    for i in range(0, len(val_predict), 288):
        plt.figure(figsize=(8, 6))
        p1, = plt.plot(x, val_predict[i:i+288], 'r', label='predict')
        p2, = plt.plot(x, val_ground_truth[i:i+288], 'b', label='target')
        plt.legend(handles=[p1, p2])
        title = df.index[i+len(predict)].split()[0]
        plt.title(title)
        if mins=='30':
            plt.savefig('figs/30/val_'+title+'.jpg')
        elif mins=='60':
            plt.savefig('figs/60/val_'+title+'.jpg')        

In [54]:
save_figs(predict30, ground_truth30, x_val30, y_val30, mins='30')

C:\Anaconda3\envs\gf\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [86]:
save_figs(predict60, ground_truth60, x_val60, y_val60, mins='60')

C:\Anaconda3\envs\gf\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [453]:

mape_loss(predict30, ground_truth30)

0.049942275818823892

In [258]:
mape_loss(predict60, ground_truth60)

0.059769881767784375

In [83]:
np.sum((np.abs(ground_truth30.flatten() - predict30.flatten()) / ground_truth30.flatten())) / len(predict30.flatten())

0.045446963417166635

In [451]:
mape_loss(y_val30.type(torch.FloatTensor).data.numpy(), model_30(x_val30)[0].type(torch.FloatTensor).data.numpy())

0.14750517385977285

In [78]:
import numpy as np

In [131]:
np.sum(np.abs((y_val30.type(torch.FloatTensor).data.numpy().flatten() - model_30(x_val30)[0].type(torch.FloatTensor).data.numpy().flatten())) / y_val30.type(torch.FloatTensor).data.numpy().flatten()) / len(y_val30)

0.049658114821822558

# LSTM

In [450]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs



if __name__ == '__main__':
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    data = torch.load('traindata.pt')
    input = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
    target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)
    test_input = Variable(torch.from_numpy(data[:3, :-1]), requires_grad=False)
    test_target = Variable(torch.from_numpy(data[:3, 1:]), requires_grad=False)
    # build the model
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
    #begin to train
    for i in range(15):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print('loss:', loss.data.numpy()[0])
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict
        future = 1000
        pred = seq(test_input, future = future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.data.numpy()[0])
        y = pred.data.numpy()
        # draw the result
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        plt.savefig('predict%d.pdf'%i)
        plt.close()